In [1]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.query import UNSET_VALUE
from datetime import datetime
import numpy as np
import uuid

In [2]:
df = pd.read_csv("../data/reddit_database.csv")

In [3]:
cluster = Cluster(["localhost"])
session = cluster.connect("reddit")

In [8]:
df["score"].describe(), df["post"].describe()

(count    545427.000000
 mean          3.927306
 std          24.602288
 min           0.000000
 25%           1.000000
 50%           1.000000
 75%           1.000000
 max        8331.000000
 Name: score, dtype: float64,
 count                                                274209
 unique                                               263705
 top       Show off the machine learning projects that yo...
 freq                                                     98
 Name: post, dtype: object)

In [10]:
for index, row in df.iterrows():
    r = {
        "id": row["id"],
        "num_comments": int(row["num_comments"]),
        "name": UNSET_VALUE,
        "author": row["author"],
        "title": row["title"],
        "selftext": row["post"] if row["post"] is not np.NaN else UNSET_VALUE,
        "subreddit": row["subreddit"],
        "score": int(row["score"]),
        "over_18": False,
        "timestamp": datetime.fromtimestamp(row["created_timestamp"]),
        "url": row["full_link"],
    }
    ps = session.prepare( 
        """
        INSERT INTO reddit.posts (
            uuid,
            num_comments,
            id,
            name,
            author,
            title,
            selftext,
            subreddit,
            score,
            over_18,
            url,
            api_timestamp
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
    )
    session.execute(
        ps, 
        (
            uuid.uuid1(),
            r["num_comments"],
            r["id"],
            r["name"],
            r["author"],
            r["title"],
            r["selftext"],
            r["subreddit"],
            r["score"],
            r["over_18"],
            r["url"],
            r["timestamp"],
        )
    )